# FNet Model for Melody Generation
This model will be trained on melodies from midi data. 

In [1]:
from os import listdir
from random import choice

from song import Song


num_of_songs = 6

midi_dir = "../examples/ghibli_dataset"
files = listdir(midi_dir)

songs = []

while num_of_songs > 0:
    s = Song(midi_dir + "/" + choice(files))
    if s.parsed:
        songs.append(s)
        num_of_songs-= 1

Loaded Song: Princess Mononoke - Ashitaka to San (C major) with 1 parts and 464 notes.
Loaded Song: Ghibli-Laputa-The-Girl-Who-Fell-From-The-Sky-Waltz with 1 parts and 1190 notes.
Loaded Song: Spirited Away - The Sixth Station (1) with 1 parts and 407 notes.
Loaded Song: Princess Mononoke - Ikiru with 1 parts and 40 notes.
Loaded Song: Owarimonogatari Second Season - Goaikyou with 3 parts and 480 notes.
Loaded Song: Ghibli-Spirited-Away-Always-With-Me with 1 parts and 790 notes.


In [2]:
# Create scale strings
from music21.stream import Part
from music21.duration import Duration
from music21.note import Note

from instrument import Instrument

base_scale = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

scales = []

base_part = Part()
for i, n in enumerate(base_scale):
    # Limitation: each scale will have the same random attributes as the base
    note = Note(n)
    note.duration = Duration(choice([1/2, 1/4, 1/8]))
    note.volume.velocity = choice(range(60,100))
    note.offset = i
    base_part.append(note)

for i in range(12):
    base_part = base_part.transpose(i)
    scales.append(Instrument(base_part))

In [3]:
# Dataset augmentation
input_seq = []
output_seq = []

s = songs[0]
print(s.parts[0].string(velocity=False))


|D4,E4,G4|1.0,1.0,1.0|0.0| |D4,E4,A4|1.0,1.0,1.0|0.0| |C2,C3|1.0,1.0|0.0| |C2|0.5|0.0| |D4,E4,A4|1.0,1.0,1.0|0.0| |G2|0.5|0.5| |E3|0.5|1.0| |G3|0.5|1.5| |A3|0.5|2.0| |G3|0.5|2.5| |C4|0.5|3.0| |A3|0.5|3.5| |C2|0.5|0.0| |D4,E4,A4|1.0,1.0,1.0|0.0| |G2|0.5|0.5| |E3|0.5|1.0| |G3|0.5|1.5| |A3|0.5|2.0| |G3|0.5|2.5| |G4,C4|1.0,1.0|3.0| |C5,A3|1.0,1.0|3.5| |C5|3.5|0.0| |C1,C2|1.0,1.0|0.0| |G2|0.5|0.5| |E3|0.5|1.0| |G3|0.5|1.5| |A3|0.5|2.0| |G3|0.5|2.5| |C4|0.5|3.0| |B4,D5,A3|1.0,1.0,1.0|3.5| |B4,D5,C2|1.0,1.0,1.0|0.0| |C5,E5,G2|1.0,1.0,1.0|0.5| |C5,E5|1.0,1.0|1.0| |E3|0.5|1.0| |G3|0.5|1.5| |A3|0.5|2.0| |G3|0.5|2.5| |C4|0.5|3.0| |A4,E5,A3|1.0,1.0,1.0|3.5| |A4,D5,G5|1.0,1.0,1.0|3.75| |C1,C2|1.0,1.0|0.0| |A4,D5,G5|1.0,1.0,1.0|0.0| |G2|0.5|0.5| |D5|2.0|1.0| |A4|3.0|1.0| |E3|0.5|1.0| |G3|0.5|1.5| |A3|0.5|2.0| |G3|0.5|2.5| |C5,C4|1.0,1.0|3.0| |D5,A3|1.0,1.0|3.5| |G4,C5,E5|1.0,1.0,1.0|0.0| |C2|0.5|0.0| |G2|0.5|0.5| |E3|0.5|1.0| |G3|0.5|1.5| |A3|0.5|2.0| |G3|0.5|2.5| |E4,E5,C4|1.0,1.0,1.0|3.0| |G4,G5,A

In [4]:
# Pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data